In [290]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd



# this will collect the url for every year page and store it in year_links
first_url = 'https://www.foxsports.com/nfl/stats?season=2019&week=101&category=RUSHING'
searching_url = first_url
url_client = urlopen(searching_url)
html_url = url_client.read()
url_client.close()
soup_url = soup(html_url, 'html.parser')




# This finds the name of all the stats and removes the first column which is just "Rushing", creates a dictionary of all the stats
stats = soup_url.table.findAll('th')
stats.pop(0)
rush_stats = ['Player Name']

for stat in stats:
    rush_stats.append(stat.text.strip())
    
stat_dict = {}
for stat in rush_stats:
    stat_dict[stat] = []



# Finds all names, reorders them, puts them in a list

names = soup_url.table.tbody.findAll('a')
name_list = []

for name in names:
    if name.span == None:
        names.remove(name)
    if name.a != None:
        names.remove(name)
        
for name in names:
    name = name.span.text
    temp = name.split()
    if len(temp) == 2:
        temp[0] = temp[0].replace(',', '')
        new = temp[1] + ' ' + temp[0]
        name_list.append(new)
    elif len(temp) == 3:
        temp[1] = temp[1].replace(',', '')
        new = temp[2] + ' ' + temp[0] + ' ' + temp[1]
        name_list.append(new)
    else:
        print(len(temp))    


        
        
# Finds and formats all stat values, long rushes are done separately due to the HTML formatting
# All stats are separated into their own lists

numbers = soup_url.table.tbody.findAll('td', {'class' : 'wisbb_priorityColumn'})


stat_vals = []

for number in numbers:
    number = number.text
    stat_vals.append(float(number))
    
att = stat_vals[::4]
yards = stat_vals[1::4]
avg = stat_vals[2::4]
td = stat_vals[3::4]

longs = soup_url.table.tbody.findAll('td', {'class' : None})
lng = []

for long in longs:
    long = float(long.text)
    lng.append(long)

In [291]:
rushing_week1_2019 = pd.DataFrame()
count = 0
for _ in range(50):
    stat_dict['Player Name'] = name_list[count]
    stat_dict['Att'] = att[count]
    stat_dict['Yds'] = yards[count]
    stat_dict['Avg'] = avg[count]
    stat_dict['TD'] = td[count]
    stat_dict['Lng'] = lng[count]
    rushing_week1_2019 = rushing_week1_2019.append(stat_dict, ignore_index = True)
    count += 1

    
    

In [292]:
rushing_week1_2019.head(50)